In [2]:
import pandas as pd
import sqlite3

In more detail:

- create a connection to the database using the library sqlite3
- using only one query for each of the groups, create two dataframes: test_results
and control_results with the columns time and avg_diff and only two rows

    - time should have the values: after and before
    - avg_diff contains the average delta among all the users for the time period
    before each of them made their first visit to the page and afterward
    - only take into account the users that have observations before and after


- we still are not using the lab ’project1’
- close the connection
- have the answer: did the hypothesis turn out to be true and the page does affect
the students’ behavior?

In [3]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

In [15]:
control_query = """
SELECT 
    'before' AS time, 
    AVG(CAST((julianday(control.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_diff
FROM control
    JOIN deadlines ON control.labname = deadlines.labs
WHERE control.labname != 'project1' 
    AND control.first_view_ts > control.first_commit_ts
    AND uid IN(SELECT uid FROM control 
               WHERE labname != 'project1' 
                     AND first_view_ts <= first_commit_ts)
UNION
SELECT 
    'after' AS time, 
    AVG(CAST((julianday(control.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_diff
FROM control
JOIN deadlines ON control.labname = deadlines.labs
WHERE control.labname != 'project1' 
    AND control.first_view_ts <= control.first_commit_ts
    AND uid IN(SELECT uid FROM control 
               WHERE labname != 'project1' 
                     AND first_view_ts > first_commit_ts);
"""
control_results = pd.io.sql.read_sql(control_query, conn)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


In [13]:
test_query = """
SELECT 'before' AS time, AVG(CAST((julianday(test.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_diff
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1' AND test.first_view_ts > test.first_commit_ts
AND uid IN(SELECT uid FROM test 
               WHERE labname != 'project1' 
                     AND first_view_ts <= first_commit_ts)
UNION
SELECT 'after' AS time, AVG(CAST((julianday(test.first_commit_ts)- julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_diff
FROM test
JOIN deadlines ON test.labname = deadlines.labs
WHERE test.labname != 'project1' AND test.first_view_ts <= test.first_commit_ts
AND uid IN(SELECT uid FROM test 
               WHERE labname != 'project1' 
                     AND first_view_ts > first_commit_ts);
"""
test_results = pd.io.sql.read_sql(test_query, conn)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [16]:
conn.close()


have the answer: did the hypothesis turn out to be true and the page does affect
the students’ behavior?  
ANSWER: Да повлияла